In [ ]:
import PublicDataReader as pdr
import pandas as pd
print(pdr.__version__)

serviceKey = 'IP3H%2BOkCGdWI49uv1bYedZoeovpC1512CuVW3QOPRtW7VJYCb22oKGjKACtsHb55i2TSn75l212DtwuS9c913w%3D%3D'

molit = pdr.Transaction(serviceKey)

In [ ]:
def get_district_code(bdongName):
    district_name = bdongName
    codeResult = molit.CodeFinder(district_name)
    district_code = codeResult.iloc[0,1]
    return district_code

In [ ]:
gu_name = ['강남구','강동구','강서구','강북구','관악구','광진구','구로구','금천구',
          '노원구','동대문구','도봉구','동작구','마포구','서대문구','성동구','성북구',
          '서초구','송파구','영등포구','용산구','양천구','은평구','종로구','중구','중랑구']

many = len(gu_name)

In [ ]:
# 구 별로 dataframe 만들기
for i in range(5,25):
    
    district_code = get_district_code(gu_name[i])
    gu_data = molit.DataCollector(molit.AptTrade, district_code, 200601, 202012)
    
    file_name = gu_name[i]
    gu_data.to_csv(f'../seoul_apt_trade_data/monthly_trade/{file_name}_trade.csv', encoding = 'cp949')